**Import**

In [ ]:
!pip install liac-arff

In [ ]:
import os
import time
import numpy as np

# torch
import torch
import torch.nn as nn
import arff

# sklearn
from sklearn.model_selection import KFold
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics import f1_score

# spen
from src.multilabel_classification.feature_network import FeatureNetwork
from src.multilabel_classification.spen_multilabel import SPENClassification
from src.multilabel_classification.utils import (
    PATH_MODELS_ML_BIB, PATH_BIBTEX, train_for_num_epochs
)
from utils_data import get_bibtex, load_data_loader
from src.multilabel_classification.feature_network import FILE_FEATURE_NETWORK

# Practical Session on Structured Prediction

This practical session is divided in two parts, each one being dedicated to a structured prediction algorithm.

- Part 1. Input output kernel regression method (IOKR)
- Part 2. Structured prediction energy network (SPEN)

**References**

[1] Belanger, David, and Andrew McCallum. "Structured prediction energy networks." International Conference on Machine Learning. PMLR, 2016.

[2] Brouard, Céline, Marie Szafranski, and Florence d'Alché-Buc. "Input output kernel regression: Supervised and semi-supervised structured output prediction with operator-valued kernels." Journal of Machine Learning Research 17 (2016).

[3] Tsochantaridis, Ioannis, et al. "Support vector machine learning for interdependent and structured output spaces." Proceedings of the twenty-first international conference on Machine learning. 2004.

[4] Katakis, Ioannis, Grigorios Tsoumakas, and Ioannis Vlahavas. "Multilabel text classification for automated tag suggestion." Proceedings of the ECML/PKDD. Vol. 18. 2008.

## Structured prediction

Recall that in structured prediction the goal is to learn a map $f: \mathcal{X} \rightarrow \mathcal{Y}$ from an input space $\mathcal{X}$ to an output space $\mathcal{Y}$, thanks to a finite set of training points $(x_i, y_i)_{i=1}^n$, that minimizes the following risk

$$\mathbb{E}_{\rho}[l(f(x), y)]$$

for a given loss $l(\hat y, y)$ defining the discrepancy between a predicted output $\hat y $ and a true output $y$, and $\rho$ is an unknown distribution on $\mathcal{X} \times \mathcal{Y}$. In structured prediction we consider **complex output** spaces $\mathcal{Y}$.

## Bibtex dataset

You are going to test the two structured prediction methods above on the the bibtex dataset. This is a standard dataset in multi-label classification. Multi-label classification can be seen as a particular instance of structured prediction where $\mathcal{Y} \subset \{0,1\}^d$.

Bibtex is a tag recommendation problem, in which the objective is to propose a relevant set of tags (e.g. url, description, journal volume) to users when they add a new Bibtex entry to the social bookmarking system Bibsonomy (see [4] for more details).

In [ ]:
# Load the bibtex dataset

Y_tr, X_tr, _, _ = get_bibtex(PATH_BIBTEX, use_train=True)
Y_te, X_te, _, _ = get_bibtex(PATH_BIBTEX, use_train=False)
n_tr = X_tr.shape[0]
n_te = X_te.shape[0]
input_dim = X_tr.shape[1]
label_dim = Y_tr.shape[1]

print(f'Train set size = {n_tr}')
print(f'Test set size = {n_te}')
print(f'Input dim. = {input_dim}')
print(f'Output dim. = {label_dim}')

**Definition (f1 score).** In this practical session we will evaluate the performances using the f1 score. In multilabel classification there are several ways to define this score. We will use the one corresponding to the sklearn function sklearn.metrics.f1_score(y, y', average="samples").

The best f1 scores which have been obtained on the Bibtex dataset are around 40%.

## I. IOKR for Structured Prediction

***

In this section, you are going to implement and test an elegant method for structured prediction called **Input Output Kernel Regression (IOKR)**. This method belongs to the family of surrogate methods for structured prediction (cf. lectures).

**Kernel methods (quick reminder).** A kernel over a space $\mathcal{Z}$ is a map $k : \mathcal{Z} \times \mathcal{Z} \rightarrow \mathbb{R}$ that can be written:

$$k(z, z') = \langle \phi(z),\, \phi(z')\rangle_{\mathcal{G}}$$

where $\mathcal{G}$ is a Hilbert space and $\phi: \mathcal{Z} \rightarrow \mathcal{G}$ a map. Hence, $k$ define a similarity between any couple of points in $\mathcal{Z}$. 

**Two examples of kernel.**

The most simple example of kernel is the linear kernel $k(z, z') = \langle z,\, z'\rangle_{\mathcal{Z}}$. So $\phi(x) = x$.

Another famous kernel is the gaussian kernel, defined, for a given $\sigma^2 >0$ by: $k(z, z') = \exp(-\frac{\|z-z\|^2}{2\sigma^2})$. Here there also exists a $\phi$ verifying the kernel definition above, but with valued in an infinite dimensional space! This is not an issue as we never need to explicitely compute any $\phi(x)$ (see below).

**Description of the method.** Consider that we are given two kernels, one over the input and another over the output space.

$$k_x(x,x') = \langle \phi(x),\, \phi(x')\rangle_{\mathcal{G}}$$

$$k_y(y,y') = \langle \psi(y),\, \psi(y')\rangle_{\mathcal{H}}$$ 

The IOKR method for structured prediction can be described in two steps:

- 1/Learning step. Learn the linear map $W : \mathcal{G} \times \mathcal{H}$ thanks to

$$\min_{W} \sum_{i=1}^n \|W\phi(x_i) - \psi(y_i)\|^2_{\mathcal{H}_y} + \lambda \|W\|^2_{F}$$

- 2/Testing step. For a given new inputs $x$ predict $\hat f(x) = \text{argmin}_{y \in \mathcal{Y}}\; \langle \psi(y),\,\hat W\phi(x) \rangle_{\mathcal{H}}$

Observe that the predictor $\hat f: \mathcal{X} \rightarrow \mathcal{Y}$ is non-linear w.r.t $x$ and also $y$.

**Algorithm.** Finally, one can derive a closed-form formula for $\hat f : \mathcal{X} \rightarrow \mathcal{Y}$ defined above. Here we do not ask you to derive this formula but we give it to you:

\begin{equation}
\hat f(x) = \text{argmin}_{y \in \mathcal{Y}} \; \alpha_x(x)^T M \alpha_y(y)
\end{equation}

with $\alpha_x(x) = \left(k_x(x, x_1), \dots, k_x(x, x_n)\right) \in \mathbb{R}^{n}$, $\alpha_y(y) = \left(k_y(y, y_1), \dots, k_y(y, y_n)\right) \in \mathbb{R}^{n}$, $K_x = \left(k_x(x_i, x_j)\right)_{i,j} \in \mathbb{R}^{n \times n}$, and $M = \left( K_x + \lambda I_{\mathbb{R}^n} \right)^{-1} \in \mathbb{R}^{n \times n}$.

![](img/IOKR.png)

**Remark (link with operator-valued kernel).** The IOKR approach is illustrated on the figure above. Observe that we perform a linear regression from $\mathcal{G}$ to $\mathcal{H}$, which can be both infinite dimensional spaces (called reproducing kernel Hilbert space) when $\phi$ and $\psi$ are defined with the non linear gaussian kernel. Moreover, notice also, that the approach corresponds to a vector-valued kernel regression from $\mathcal{X}$ to $\mathcal{H}$. This situation can be interpreted through the use of an operator-valued kernel (ovk). Here we use the ovk defined by $\tilde k_x(x,x') =  k_x(x,x') I_{\mathcal{H}}$, where $I_{\mathcal{H}}$ denotes the identity operator of $\mathcal{H}$.

***


#### Question 1. (decoding) 
During the decoding step, for a given input $x$ one needs to compute the scores $s(x,y) = \alpha_x(x)^T M \alpha_y(y)$ for all $y \in \mathcal{Y}$ in order to predict $y$ with the maximum $s(x,y)$. Considering all the $\{0,1\}^{159}$ possible 0-1 vectors is too costly. Hence, here we consider only points in the training set Y_tr as possible candidates.

Show that, for any test set X_te of size $n_{te}$, you can compute all the $n \times n_{te}$ scores, by only making matrix multiplications of the three following matrices: $M \in \mathbb{R}^{n \times n}$ (defined above), $K_x = \left(k_x(x_i, x_j)\right)_{i,j} \in \mathbb{R}^{n \times n_{te}}$, and $K_y = \left(k_y(y_i, y_j)\right)_{i,j} \in \mathbb{R}^{n \times n}$.

#### Question 2. (implementation)
Implement the method IOKR using gaussian input and output kernels, with parameters $\sigma_x^2, \sigma_y^2 >0$ respectively. Define a python class with a method fit(X_tr, Y_tr, lambda, sigmax, sigmay), and a method predict(X_te). Use the rbf_kernel python function loaded above to compute the gram matrices $K_x, K_y$. 

In [ ]:
# Implementation

class IOKR:
    
    def __init__(self):
        self.X_tr = None
        self.Y_tr = None
        self.Ky = None
        self.sy = None
        self.M = None
        self.verbose = 0
        self.linear = False
        
    def fit(self, X, Y, L, sx, sy):
        
        pass
    
    def predict(self, X_te):
        
        pass

#### Question 3. (sanity check)
Verify that your algorithm is able to learn a predictor that overfits the training set, i.e. such that the f1 score on the train set is close to $1$. For instance, plot the train f1 score w.r.t $\lambda$ (for $s_x, s_y$ big enough).

#### Question 4. (hyperparameters selection)

We would like to assess the performance of IOKR on the bibtex dataset.

In order to select the hyperparameters $\lambda, \sigma_x^2, \sigma_y^2$, proceed as follows. For a given tuples of parameters, train with $\frac{4}{5}$ of the train set, and then compute the f1 score (called validation score) on the remaining $\frac{1}{5}$ of the train set (called validation set). By repeating this over a grid of hyperparameters select the best tuples with the highest validation score.

Explain the role of the validation set. Why it is extracted from the train set? Then, compute the test f1 score on the test set, using the best selected parameters.

#### Question 5. (linear vs non linear)

Modify your code such that you can use IOKR with an output linear kernel $k_y(y,y') = \langle y, y' \rangle_{\mathbb{R}^d}$. Then, compare the performance of the linear and the gaussian (output) kernels on the bibtex dataset. For this purpose, you need to select the hyper-parameters ($\lambda, \sigma_x^2$ in the case of the linear output kernel, and $\lambda, \sigma_x^2, \sigma_y^2$ for the gaussian kernel).

#### Question 6. (computational complexity)

What is the complexity in time and space of IOKR (for training and decoding)?

## II. Structured prediction energy network (SPEN)

Structured Prediction Energy Networks (SPENs) are deep energy-based models, constituting a flexible framework for structured prediction, in particular multi-label classification.

SPENs tackle multi-label classification, then $\mathcal{Y} = \{0,1\}^L$ with $L \in \mathbb{N}^*$. Thus, for a given input $x$, a SPEN performs prediction by solving the following problem::
$$
    \underset{y}{\operatorname{min}} ~ E_x(y) \quad \text{subject to} \quad y \in \{0,1\}^L, \quad \text{(1)}
$$
where enery function $E_x$ is encoded by a deep architecture.\\
Authors choose to solve an easier problem and optimize over a convex relaxation of the constraint set:
$$
    \underset{\bar{y}}{\operatorname{min}} ~ E_x(\bar{y}) \quad \text{subject to} \quad \bar{y} \in [0,1]^L. \quad \text{(2)}
$$
Optimization over $[0,1]^L$ to obtain a prediction $\bar{y}^* \in [0,1]^L$ can be performed using projected gradient descent or entropic mirror descent. Finally, for an input $x$, prediction $f(x) = (f(x)_1, \ldots, f(x)_L)^\top \in \{0,1\}^L$ is obtained by rounding each coordinate $\bar{y}_i^*$ to $0$ or $1$. Hence, for all $1 \leq i \leq L$:
$$
    f(x)_i = \left\{
            \begin{array}{ll}
                 0 & \mbox{if } \bar{y}_i^* < 0.5\\
                1 & \mbox{otherwise.}
            \end{array}
            \right.
$$
Where $\bar{y}^* = \underset{\bar{y} \in [0,1]^L}{\operatorname{argmin}} ~ E_x(\bar{y})$.

A SPEN parameterizes $E_x(\bar{y})$ as a neural network that takes both $x$ and $\bar{y}$ as inputs and returns the energy. It consists of two deep architectures as illustrated in the following image:

- a feature extraction network;
- an energy network.

![A SPEN architecture](img/Spen_architecture.png)

This practical session is divided in two sections:

- A) Learning feature extraction network;
- B) Learning energy network.

We used an implementation of SPEN in python with PyTorch by Philippe Beardsell and Chih-Chao Hsu (cf. https://github.com/philqc/deep-value-networks-pytorch). Small changes have been made.

### A) Feature extraction network

In this section, the goal is to learn the feature extractor part of a SPEN architecture encoded by this deep network:
$$
    F(x) = g(A_2g(A_1x)). \quad \text{(3)}
$$
Hence, by gradient descent and backpropagation we learn the parameters $A_1$ and $A_2$.
This training is performed by adding a third layer (which is used only during training and removed later) with a sigmoid activation function and performing a classification task with a Binary Cross Entropy loss.
The optimizer used can be rather "adam" or "sgd" solver.
The rest of hyperparameters are indicated below.

#### Question 7.  (learning feature extractor network)

Use functions train_feature_extraction and test_feature_extraction to train and test feature extraction network and study the influence of optimisation hyperparameters (learning rate, momentum, batch size and number of epochs).

In [ ]:
# optimization solver's parameters

lr = 1e-5
momentum = 0.1

In [ ]:
# Feature extractor network instantiation

f_net = FeatureNetwork(lr=lr, momentum=momentum,
                       optimizer="adam", weight_decay=0,
                       input_dim=input_dim, label_dim=label_dim)

In [ ]:
# function for training the feature extractor

def train_feature_extraction(X_tr, Y_tr, X_val, Y_val,
                             f_net: FeatureNetwork,
                             path_save: str, n_epochs: int,
                             batch_size: int,
                             step_size_scheduler: int,
                             norm_inputs=True):
    """
    Trains a feature extractor network
    """
    
    use_cuda = torch.cuda.is_available()
    
    train_loader = load_data_loader(X_tr, Y_tr, use_cuda,
                                    batch_size=batch_size,
                                    norm_inputs=norm_inputs,
                                    shuffle=False)
    
    valid_loader = load_data_loader(X_val, Y_val, use_cuda,
                                    batch_size=batch_size,
                                    norm_inputs=norm_inputs,
                                    shuffle=False)

    scheduler = torch.optim.lr_scheduler.StepLR(
        f_net.optimizer, step_size=step_size_scheduler, gamma=0.1
    )

    loss_train, loss_valid, list_f1_valid = train_for_num_epochs(
        f_net,
        train_loader,
        valid_loader,
        os.path.join(path_save, FILE_FEATURE_NETWORK),
        n_epochs,
        scheduler
    )
    
    return loss_train, loss_valid, list_f1_valid

In [ ]:
# Split the training set in train/validation sets

train_valid_ratio=0.8

n_train = int(len(X_tr) * train_valid_ratio)
indices = list(range(len(X_tr)))

x_train, y_train = X_tr[:n_train], Y_tr[:n_train]
x_valid, y_valid = X_tr[n_train:], Y_tr[n_train:]

In [ ]:
# Number of epochs and batch size settings

n_epochs = 10
batch_size = 32

In [ ]:
# feature extractor training

loss_train, loss_valid, list_f1_valid = train_feature_extraction(x_train, y_train,
                                                                 x_valid, y_valid,
                                                                 f_net,
                                                                 PATH_MODELS_ML_BIB,
                                                                 n_epochs=n_epochs,
                                                                 batch_size=batch_size,
                                                                 step_size_scheduler=20)

In [ ]:
# function for testing the feature extractor

def test_feature_extractor(X_te, Y_te, path_save, f_net):
    """
    Tests a feature extractor network
    """
    use_cuda = torch.cuda.is_available()

    test_loader = load_data_loader(X_te, Y_te, use_cuda)

    print('Computing the F1 Score on the test set...')
    test_loss, test_mean_f1 = f_net.valid(test_loader)
    
    return test_loss, test_mean_f1

In [ ]:
# compute the test f1 score of the feature extractor network

test_loss, test_mean_f1 = test_feature_extractor(X_te, Y_te, PATH_MODELS_ML_BIB, f_net)

### B) Energy network

Now, the goal is to learn the energy network part of a SPEN architecture encoded by this deep network:
$$
    E_x(\bar{y}) = E(F(x),\bar{y}) = E_{x}^{\text {local }}(\bar{y}) + E_{x}^{\text {label }}(\bar{y}) \quad \text{(4)}
$$
where:
$$
    E_{x}^{\text {local }}(\bar{y})=\sum_{i=1}^{L} \bar{y}_{i} b_{i}^{\top} F(x) \quad \text{(5)}
$$
and
$$
    E_{x}^{\text {label }}(\bar{y})=c_{2}^{\top} g\left(C_{1} \bar{y}\right) \quad \text{(6)}
$$
and $F(x)$ is the feature extractor previously trained (with only 2 layers, the third one is now removed as said earlier).
Hence, parameers to learn are $b_i$s, $C_1$ and $c_2$.

To achieve learning, authors use a structured Hinge loss, similarly to Structured Support Vector Machine (SSVM) (see Lecture 1).

Then, they define $\Delta\left(y_{p}, y_{g}\right)$ to be an error function between a prediction $y_{p}$ and the ground truth $y_{g}$ not dependent on the structured output set, such as the Hamming loss. Let $[\cdot]_{+}=\max (0, \cdot) .$ The SSVM minimizes:
$$
    \sum_{\left\{x_{i}, y_{i}\right\}} \max _{y}\left[\Delta\left(y_{i}, y\right)-E_{x_{i}}(y)+E_{x_{i}}\left(y_{i}\right)\right]_{+}. \quad \text{(7)}
$$
 Learning is performed by minimizing the loss with respect to the parameters of the deep architecture $E_{x}$ using mini-batch stochastic gradient descent. For $\left\{x_{i}, y_{i}\right\}$, computing the subgradient of (7) with respect to the prediction requires loss-augmented inference:
$$
     y_{p}=\underset{y}{\arg \min }\left(-\Delta\left(y_{i}, y\right)+E_{x_{i}}(y)\right). \quad \text{(8)}
$$
With this, the subgradient of (7) with respect to the model parameters is obtained by back-propagation through $E_{x}$.

#### Question 8.  (role of $E_x^{label}$)

What is the role of $E_x^{label}$? Describe breifly its specificity and difference from $E_x^{local}$.

#### Question 9.  (learning energy network)

Once the feature extraction network is trained, use functions train_energy_network and test_energy_network to train and test energy network and study the influence of :

- The size of $c_2$: num_pairwise.
- optimisation hyperparameters for network learning (optimisation part associated to learning $b_i$s, $C_1$ and $c_2$): learning rate, momentum, batch size and number of epochs;
- optimisation hyperparameters for inference (computation of $\bar y$): learning rate and momentum.

In [ ]:
# optimization solver's parameters

learning_rate = 1e-5
momentum = 0.90

In [ ]:
# Optimisation hyperparameters for loss-augmented inference

inf_lr = 0.1
momentum_inf = 0.95

In [ ]:
# num_pairwise x dim label

num_pairwise = 16

In [ ]:
# Energy network instantiation

feature_extractor_file = "saved_results/bibtex/feature_network.pth"

spen = SPENClassification(feature_extractor_file, # Loading feature extractor trained earlier
                          loss_fn="mse",
                          input_dim=input_dim,
                          label_dim=label_dim,
                          optim="adam",
                          learning_rate=learning_rate,
                          momentum=momentum,
                          non_linearity=nn.Softplus(),
                          num_pairwise=num_pairwise
                         )

In [ ]:
# function for training SPEN

def train_energy_network(X_tr, Y_tr, X_val, Y_val,
                         path_save, spen,
                         n_epochs=10, batch_size=32,
                         norm_inputs=True):
    """
    Trains an energy network
    """
    use_cuda = torch.cuda.is_available()

    train_loader = load_data_loader(X_tr, Y_tr, use_cuda,
                                    batch_size=batch_size,
                                    norm_inputs=norm_inputs,
                                    shuffle=False)
    
    valid_loader = load_data_loader(X_val, Y_val, use_cuda,
                                    batch_size=batch_size,
                                    norm_inputs=norm_inputs,
                                    shuffle=False)

    input_dim = X_tr.shape[1]
    label_dim = Y_tr.shape[1]

    name = 'spen_bibtex'
    path_save_model = os.path.join(path_save, name + '.pth')

    scheduler = torch.optim.lr_scheduler.StepLR(spen.optimizer, step_size=10, gamma=0.1)
    n_epochs = n_epochs

    loss_train, loss_valid, list_f1_valid = train_for_num_epochs(spen,
                                                                 train_loader,
                                                                 valid_loader,
                                                                 path_save_model,
                                                                 n_epochs,
                                                                 scheduler)
    
    return loss_train, loss_valid, list_f1_valid

In [ ]:
# Number of epochs and batch size settings

n_epochs = 10
batch_size = 32

In [ ]:
# SPEN training

loss_train, loss_valid, list_f1_valid = train_energy_network(x_train, y_train,
                                                             x_valid, y_valid,
                                                             PATH_MODELS_ML_BIB,
                                                             spen, n_epochs=n_epochs,
                                                             batch_size=batch_size,
                                                             norm_inputs=True)

In [ ]:
# function for testing SPEN

def test_energy_network(X_te, Y_te, path_save, spen):
    """
    Tests an energy network
    """
    use_cuda = torch.cuda.is_available()

    test_loader = load_data_loader(X_te, Y_te, use_cuda)

    print('Computing the F1 Score on the test set...')
    test_loss, test_mean_f1 = spen.valid(test_loader)
    
    return test_loss, test_mean_f1

In [ ]:
# Test of SPEN on the test set

test_loss, test_mean_f1 = test_energy_network(X_te, Y_te, PATH_MODELS_ML_BIB, spen)

#### Question 10.  ( structured network)

Is the structured part of the SPEN's model $E^{label}_x$ useful with the Bibtex dataset? What can you conclude? 